In [1]:
# Import dependencies
# Standard python libraries
import os
# Third-party libraries
from flask import Flask, redirect, request, url_for, render_template, session
import pandas as pd

# Internal imports
from db import get_df
from mfl import get_mfl_league, get_mfl_liveScoring, get_mfl_projectedScores

# Find environment variables
DATABASE_URL = os.environ.get("DATABASE_URL", None)
# sqlalchemy deprecated urls which begin with "postgres://"; now it needs to start with "postgresql://"
if DATABASE_URL.startswith("postgres://"):
    DATABASE_URL = DATABASE_URL.replace("postgres://", "postgresql://", 1)

user_league = "53906"
week = "2"
matchNumber = 0

In [2]:
# Get required data
leagueDF = get_mfl_league(user_league)
liveDF = get_mfl_liveScoring(user_league)
projDF = get_mfl_projectedScores(user_league, week)
players = get_df("player_df")
players = players.rename(columns={"PlayerID":"id_mfl"})

In [3]:
# Merge datasets
df = players.merge(
    projDF, on="id_mfl", how="left"
).merge(
    liveDF, on="id_mfl", how="left"
).merge(
    leagueDF, on="franchiseID", how="left"
)

In [4]:
# Reorder columns
df = df[["matchup", "franchiseName", "status", "Name", "Position", "Team", "sharkProjection", "liveScore", "secondsRemaining"]]

In [5]:
# Clean df
df.dropna(inplace=True)
# Define data types
df['sharkProjection'] = df['sharkProjection'].astype('float32', copy=False)
df['liveScore'] = df['liveScore'].astype('float32', copy=False)
df['secondsRemaining'] = df['secondsRemaining'].astype('int', copy=False)
df['matchup'] = df['matchup'].astype('int', copy=False)


In [6]:
# Calculate final Projections based on amount of time remaining
def calcScore(row):
    # Use a different calculation method for defenses since defenses do not accrue points
    if row["Position"] != "DF":
        result = row['liveScore'] + (row['sharkProjection'] * row['secondsRemaining'] / 3600)
    else:
        result = (row['sharkProjection'] * row['secondsRemaining'] + row['liveScore'] * (3600 - row['secondsRemaining'])) / 3600
    return result
df['finalProjection'] = df.apply(calcScore, axis=1)
df

,matchup,franchiseName,status,Name,Position,Team,sharkProjection,liveScore,secondsRemaining,finalProjection
0,4,O'Henry's Stiffies,starter,"Taylor, Jonathan",RB,IND,22.400000,0.0,3600,22.400000
1,5,Comeback Kings,starter,"Allen, Josh",QB,BUF,24.200001,0.0,3600,24.200001
2,5,IDK much about soccer,starter,"Ekeler, Austin",RB,LAC,16.799999,13.1,0,13.100000
3,0,Saskatoon Squatches,starter,"Mahomes, Patrick",QB,KCC,23.600000,16.6,0,16.600000
4,1,Washington Fantasy Football Team,starter,"Kupp, Cooper",WR,LAR,16.299999,0.0,3600,16.299999
...,...,...,...,...,...,...,...,...,...,...
556,3,Gus The Bus,nonstarter,"Pierce, Dameon",RB,HOU,4.100000,0.0,3600,4.100000
652,0,Saskatoon Squatches,nonstarter,"Warren, Jaylen",RB,PIT,1.700000,0.0,3600,1.700000
714,0,Crocodilopolis Body Snatchers,nonstarter,"Williams, Avery",RB,ATL,1.100000,0.0,3600,1.100000
948,4,O'Henry's Stiffies,starter,"Dortch, Greg",WR,ARI,1.100000,0.0,3600,1.100000


In [34]:
# Create matchup summary table
starts = df.loc[df['status']=="starter"]
matchSumm = starts.groupby(['matchup',"franchiseName"])['finalProjection'].sum()
matchSumm = pd.DataFrame(matchSumm)
matchSumm = matchSumm.reset_index()
matchSumm.loc[matchSumm.index%2==0, 'pivotIndex'] = "A" 
matchSumm.loc[matchSumm.index%2==1, 'pivotIndex'] = "B"
matchSumm = matchSumm[['matchup', 'pivotIndex', 'franchiseName', 'finalProjection']]
#matchSumm = matchSumm.set_index('matchup')
matchSumm = matchSumm.reset_index(drop=True)
matchSumm

,matchup,pivotIndex,franchiseName,finalProjection
0,0,A,Crocodilopolis Body Snatchers,180.899999
1,0,B,Saskatoon Squatches,166.400000
2,1,A,Pretty Big Wieners,156.500002
3,1,B,Washington Fantasy Football Team,168.100000
4,2,A,Down with the Sickness unto Death,164.099998
5,2,B,Verdansk Vaqueros FFC,149.800000
6,3,A,Gus The Bus,155.900001
7,3,B,Providence Monastic Spirits,144.999999
8,4,A,O'Henry's Stiffies,153.400001
9,4,B,WHAM! BAM! Thank You CAM!,164.900000


In [ ]:
# M0FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M0PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']
# M1FA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'franchiseName']
# M1PA = matchSumm.loc[matchSumm.matchup==0].loc[matchSumm.pivotIndex=="A", 'finalProjection']

In [13]:
# matchhead = pd.DataFrame()
# for matchNo in matchSumm.matchup.unique():
#     dfM = matchSumm.loc[matchSumm.matchup==matchNo]
#     dfM.columns = ['matchup', 'franchiseName', 'finalProjection']
#     matchhead = pd.concat([matchhead, dfM], axis=1)
# matchhead.drop(columns=['matchup', 'pivotIndex'], inplace=True)


In [45]:
M1A = matchSumm.loc[matchSumm.matchup==0].drop(columns=['matchup', 'pivotIndex'])
M2A = matchSumm.loc[matchSumm.matchup==1].drop(columns=['matchup', 'pivotIndex'])
M3A = matchSumm.loc[matchSumm.matchup==2].drop(columns=['matchup', 'pivotIndex'])
M4A = matchSumm.loc[matchSumm.matchup==3].drop(columns=['matchup', 'pivotIndex'])
M5A = matchSumm.loc[matchSumm.matchup==4].drop(columns=['matchup', 'pivotIndex'])
M6A = matchSumm.loc[matchSumm.matchup==5].drop(columns=['matchup', 'pivotIndex'])

In [46]:
M1A

,franchiseName,finalProjection
0,Crocodilopolis Body Snatchers,180.899999
1,Saskatoon Squatches,166.400000


In [58]:
#Get franchise Names based on which matchup is selected
franchiseA = matchSumm.loc[(matchSumm.matchup==0) & (matchSumm.pivotIndex=="A"), 'franchiseName'][matchNumber]
franchiseB = matchSumm.loc[(matchSumm.matchup==0) & (matchSumm.pivotIndex=="B"), 'franchiseName'][matchNumber]


In [91]:
# Select players on franchise roster
tableA = df.loc[df.franchiseName==franchiseA]
tableA.reset_index(inplace=True, drop=True)
# Create summary row
rowHead = ["summary", franchiseA, "Total", "", "", ""]
rowSummary = rowHead + list(tableA.loc[tableA.status=='starter'].sum()[['sharkProjection', 'liveScore', 'secondsRemaining', 'finalProjection']])
tableA.loc[len(tableA)] = rowSummary
# Create categories to sort by
tableA.status = pd.Categorical(tableA.status, 
                      categories=["nonstarter", 'Total', "starter"],
                      ordered=True)
tableA.Position = pd.Categorical(tableA.Position, 
        categories=['Def', 'PK', 'TE', 'WR', 'RB', 'QB', ""],
        ordered=True)
# Sort
tableA.sort_values(['status', 'Position', 'finalProjection'], inplace=True, ascending=False)
tableA.reset_index(inplace=True, drop=True)
# Rename columns
tableA.rename(columns={})
tableA

/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/indexing.py:723: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  iloc._setitem_with_indexer(indexer, value, self.name)
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/core/generic.py:5516: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  self[name] = value
/Users/travisharry/opt/anaconda3/envs/ffdemo/lib/python3.7/site-packages/pandas/util/_decorators.py:311: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See

,matchup,franchiseName,status,Name,Position,Team,sharkProjection,liveScore,secondsRemaining,finalProjection
0,0,Crocodilopolis Body Snatchers,starter,"Murray, Kyler",QB,ARI,21.799999,0.0,3600,21.799999
1,0,Crocodilopolis Body Snatchers,starter,"Hurts, Jalen",QB,PHI,20.600000,0.0,3600,20.600000
2,0,Crocodilopolis Body Snatchers,starter,"Mixon, Joe",RB,CIN,17.299999,0.0,3600,17.299999
3,0,Crocodilopolis Body Snatchers,starter,"Jacobs, Josh",RB,LVR,14.100000,0.0,3600,14.100000
4,0,Crocodilopolis Body Snatchers,starter,"Montgomery, David",RB,CHI,13.300000,0.0,3600,13.300000
5,0,Crocodilopolis Body Snatchers,starter,"Brown, Marquise",WR,ARI,10.300000,0.0,3600,10.300000
6,0,Crocodilopolis Body Snatchers,starter,"Moore, D.J.",WR,CAR,9.400000,0.0,3600,9.400000
7,0,Crocodilopolis Body Snatchers,starter,"Duvernay, Devin",WR,BAL,8.300000,0.0,3600,8.300000
8,0,Crocodilopolis Body Snatchers,starter,"McKenzie, Isaiah",WR,BUF,8.300000,0.0,3600,8.300000
9,0,Crocodilopolis Body Snatchers,starter,"Knox, Dawson",TE,BUF,7.500000,0.0,3600,7.500000


In [ ]:
# Select players on franchise roster
tableB = df.loc[df.franchiseName==franchiseB]
tableB.reset_index(inplace=True, drop=True)
# Create summary row
rowHead = ["summary", franchiseA, "Total", "", "", ""]
rowSummary = rowHead + list(tableB.loc[tableB.status=='starter'].sum()[['sharkProjection', 'liveScore', 'secondsRemaining', 'finalProjection']])
tableB.loc[len(tableB)] = rowSummary
# Create categories to sort by
tableB.status = pd.Categorical(tableB.status, 
                      categories=["nonstarter", 'Total', "starter"],
                      ordered=True)
tableB.Position = pd.Categorical(tableB.Position, 
        categories=['Def', 'PK', 'TE', 'WR', 'RB', 'QB', ""],
        ordered=True)
# Sort
tableB.sort_values(['status', 'Position', 'finalProjection'], inplace=True, ascending=False)
tableB.reset_index(inplace=True, drop=True)
# Rename columns
tableB.rename(columns={})
tableB